### Preprocesamiento

In [ ]:
import pandas as pd
from tqdm import tqdm

In [3]:
path = 'final_data'
cols = ['danceability', 'energy', 'key', 'loudness', 'mode',
'speechiness', 'acousticness', 'instrumentalness', 'liveness',
'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url',
'duration_ms', 'time_signature']
all_files = glob.glob(os.path.join(path, "*.csv"))
df = pd.concat((pd.read_csv(f, names=cols) for f in all_files), ignore_index=True)


In [4]:
df = df.drop_duplicates(subset='id').drop(["type", 'uri', 'track_href', 'analysis_url'], axis=1).dropna()

In [5]:
df.to_csv('data/audio_features.csv')

In [5]:
df = pd.read_csv('data/audio_features.csv')
df_playlists = pd.read_csv('data/playlist.tsv', sep='\t')
df_playlists['track_uris'] = df_playlists['track_uris'].apply(eval)


/tmp/ipykernel_124678/848576910.py:1: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/audio_features.csv')


In [7]:
all_tracks = []
for i in df_playlists['track_uris']:
    all_tracks.extend(i)
valid_tracks_ = set(df['id'].unique())
valid_tracks = {i: i in valid_tracks_ for i in all_tracks}

In [8]:
def filter_playlists(playlist, valid_tracks):
    playlist = [track for track in playlist if valid_tracks[track]]
    return str(playlist)

In [9]:
tqdm.pandas()
df_playlists['track_uris'] = df_playlists['track_uris'].progress_apply(lambda x: filter_playlists(x, valid_tracks))
df_playlists = df_playlists[df_playlists['track_uris'].apply(lambda x: len(eval(x)) > 1)]
df_playlists.to_csv('data/filtered_playlists.csv', index=False, sep='\t')

100%|██████████| 1000000/1000000 [00:18<00:00, 53345.45it/s]
